## Linearna i Polinomna Regresija za Određivanje Cijena Bundeva - Lekcija 3
<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografika: Dasani Madipalli</figcaption>


<!--![Infografika: Dasani Madipalli](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->

#### Uvod

Do sada ste istražili što je regresija koristeći uzorke podataka prikupljenih iz skupa podataka o cijenama bundeva, koji ćemo koristiti tijekom ove lekcije. Također ste ih vizualizirali pomoću `ggplot2`. 💪

Sada ste spremni dublje zaroniti u regresiju za strojno učenje. U ovoj lekciji naučit ćete više o dvije vrste regresije: *osnovnoj linearnoj regresiji* i *polinomnoj regresiji*, zajedno s nekim matematičkim osnovama ovih tehnika.

> Tijekom ovog kurikuluma pretpostavljamo minimalno matematičko predznanje i nastojimo ga učiniti pristupačnim za studente iz drugih područja, pa obratite pažnju na bilješke, 🧮 napomene, dijagrame i druge alate za učenje koji će vam pomoći u razumijevanju.

#### Priprema

Podsjetnik: učitavate ove podatke kako biste mogli postavljati pitanja o njima.

- Kada je najbolje vrijeme za kupnju bundeva?

- Koju cijenu mogu očekivati za kutiju minijaturnih bundeva?

- Trebam li ih kupiti u košarama od pola bušela ili u kutijama od 1 1/9 bušela? Nastavimo istraživati ove podatke.

U prethodnoj lekciji kreirali ste `tibble` (modernu reinterpretaciju okvira podataka) i popunili ga dijelom izvornog skupa podataka, standardizirajući cijene prema bušelu. Međutim, na taj način uspjeli ste prikupiti samo oko 400 podataka i to samo za jesenske mjesece. Možda možemo dobiti malo više detalja o prirodi podataka ako ih dodatno očistimo? Vidjet ćemo... 🕵️‍♀️

Za ovaj zadatak trebat će nam sljedeći paketi:

- `tidyverse`: [tidyverse](https://www.tidyverse.org/) je [zbirka R paketa](https://www.tidyverse.org/packages) osmišljena kako bi znanost o podacima bila brža, lakša i zabavnija!

- `tidymodels`: [tidymodels](https://www.tidymodels.org/) okvir je [zbirka paketa](https://www.tidymodels.org/packages/) za modeliranje i strojno učenje.

- `janitor`: [janitor paket](https://github.com/sfirke/janitor) pruža jednostavne alate za pregled i čišćenje neurednih podataka.

- `corrplot`: [corrplot paket](https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html) nudi vizualni alat za istraživanje korelacijskih matrica koji podržava automatsko preuređivanje varijabli kako bi se otkrili skriveni obrasci među varijablama.

Možete ih instalirati pomoću:

`install.packages(c("tidyverse", "tidymodels", "janitor", "corrplot"))`

Skripta ispod provjerava imate li instalirane potrebne pakete za dovršetak ovog modula i instalira ih za vas ako nedostaju.


In [ ]:
suppressWarnings(if (!require("pacman")) install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, corrplot)

Kasnije ćemo učitati ove sjajne pakete i učiniti ih dostupnima u našoj trenutnoj R sesiji. (Ovo je samo za ilustraciju, `pacman::p_load()` je to već učinio za vas)

## 1. Linija linearne regresije

Kao što ste naučili u Lekciji 1, cilj vježbe linearne regresije je moći nacrtati *liniju* *najboljeg* *prilagođavanja* kako bi se:

-   **Prikazali odnosi između varijabli**. Prikazali odnos između varijabli.

-   **Napravile predikcije**. Napravile točne predikcije o tome gdje bi novi podatkovni točkasti element mogao pasti u odnosu na tu liniju.

Za crtanje ove vrste linije koristimo statističku tehniku zvanu **Regresija najmanjih kvadrata**. Pojam `najmanji kvadrati` znači da su sve podatkovne točke oko regresijske linije kvadrirane i zatim zbrojene. Idealno, taj konačni zbroj je što manji, jer želimo mali broj pogrešaka, odnosno `najmanje kvadrate`. Kao takva, linija najboljeg prilagođavanja je linija koja nam daje najnižu vrijednost za zbroj kvadriranih pogrešaka - otuda naziv *regresija najmanjih kvadrata*.

To radimo jer želimo modelirati liniju koja ima najmanju kumulativnu udaljenost od svih naših podatkovnih točaka. Također kvadriramo članove prije zbrajanja jer nas zanima njihova veličina, a ne smjer.

> **🧮 Pokaži mi matematiku**
>
> Ova linija, nazvana *linija najboljeg prilagođavanja*, može se izraziti [jednadžbom](https://en.wikipedia.org/wiki/Simple_linear_regression):
>
>     Y = a + bX
>
> `X` je '`objašnjavajuća varijabla` ili `prediktor`'. `Y` je '`ovisna varijabla` ili `ishod`'. Nagib linije je `b`, a `a` je presjek na osi y, koji se odnosi na vrijednost `Y` kada je `X = 0`.
>

> ![](../../../../../../2-Regression/3-Linear/solution/images/slope.png "nagib = $y/x$")
    Infografika: Jen Looper
>
> Prvo, izračunajte nagib `b`.
>
> Drugim riječima, i referirajući se na originalno pitanje o podacima o bundevama: "predvidite cijenu bundeve po bušelu prema mjesecu", `X` bi se odnosio na cijenu, a `Y` na mjesec prodaje.
>
> ![](../../../../../../2-Regression/3-Linear/solution/images/calculation.png)
    Infografika: Jen Looper
> 
> Izračunajte vrijednost Y. Ako plaćate oko 4 dolara, mora da je travanj!
>
> Matematika koja izračunava liniju mora pokazati nagib linije, koji također ovisi o presjeku, odnosno gdje se `Y` nalazi kada je `X = 0`.
>
> Metodu izračuna ovih vrijednosti možete vidjeti na web stranici [Math is Fun](https://www.mathsisfun.com/data/least-squares-regression.html). Također posjetite [ovaj kalkulator najmanjih kvadrata](https://www.mathsisfun.com/data/least-squares-calculator.html) kako biste vidjeli kako vrijednosti brojeva utječu na liniju.

Nije tako strašno, zar ne? 🤓

#### Korelacija

Još jedan pojam koji treba razumjeti je **Koeficijent korelacije** između danih X i Y varijabli. Koristeći dijagram raspršenja, možete brzo vizualizirati ovaj koeficijent. Dijagram s podatkovnim točkama uredno raspoređenim u liniji ima visoku korelaciju, dok dijagram s podatkovnim točkama razbacanim svuda između X i Y ima nisku korelaciju.

Dobar model linearne regresije bit će onaj koji ima visok (bliži 1 nego 0) Koeficijent korelacije koristeći metodu Regresije najmanjih kvadrata s regresijskom linijom.


## **2. Ples s podacima: stvaranje podatkovnog okvira za modeliranje**

<p >
   <img src="../../images/janitor.jpg"
   width="700"/>
   <figcaption>Ilustracija: @allison_horst</figcaption>


<!--![Ilustracija: \@allison_horst](../../../../../../2-Regression/3-Linear/images/janitor.jpg){width="700"}-->


Učitajte potrebne biblioteke i skup podataka. Pretvorite podatke u podatkovni okvir koji sadrži podskup podataka:

-   Uzmite samo bundeve čija je cijena izražena po bušelu

-   Pretvorite datum u mjesec

-   Izračunajte cijenu kao prosjek visokih i niskih cijena

-   Pretvorite cijenu tako da odražava cijenu po količini bušela

> Ove korake smo obradili u [prethodnoj lekciji](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/2-Data/solution/lesson_2-R.ipynb).


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)
library(lubridate)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n = 5)

U duhu čiste avanture, istražimo [`janitor package`](../../../../../../2-Regression/3-Linear/solution/R/github.com/sfirke/janitor) koji pruža jednostavne funkcije za pregled i čišćenje neurednih podataka. Na primjer, pogledajmo nazive stupaca za naše podatke:


In [ ]:
# Return column names
pumpkins %>% 
  names()

🤔 Možemo bolje. Hajdemo napraviti ove nazive stupaca `friendR` tako da ih pretvorimo u [snake_case](https://en.wikipedia.org/wiki/Snake_case) konvenciju koristeći `janitor::clean_names`. Za više informacija o ovoj funkciji: `?clean_names`


In [ ]:
# Clean names to the snake_case convention
pumpkins <- pumpkins %>% 
  clean_names(case = "snake")

# Return column names
pumpkins %>% 
  names()

Puno tidyR-a 🧹! Sada, ples s podacima koristeći `dplyr` kao u prethodnoj lekciji! 💃


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(variety, city_name, package, low_price, high_price, date)



# Extract the month from the dates to a new column
pumpkins <- pumpkins %>%
  mutate(date = mdy(date),
         month = month(date)) %>% 
  select(-date)



# Create a new column for average Price
pumpkins <- pumpkins %>% 
  mutate(price = (low_price + high_price)/2)


# Retain only pumpkins with the string "bushel"
new_pumpkins <- pumpkins %>% 
  filter(str_detect(string = package, pattern = "bushel"))


# Normalize the pricing so that you show the pricing per bushel, not per 1 1/9 or 1/2 bushel
new_pumpkins <- new_pumpkins %>% 
  mutate(price = case_when(
    str_detect(package, "1 1/9") ~ price/(1.1),
    str_detect(package, "1/2") ~ price*2,
    TRUE ~ price))

# Relocate column positions
new_pumpkins <- new_pumpkins %>% 
  relocate(month, .before = variety)


# Display the first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Dobar posao! 👌 Sada imate čist i uredan skup podataka na kojem možete izgraditi svoj novi regresijski model!

Što kažete na scatter plot?


In [ ]:
# Set theme
theme_set(theme_light())

# Make a scatter plot of month and price
new_pumpkins %>% 
  ggplot(mapping = aes(x = month, y = price)) +
  geom_point(size = 1.6)


Raspršeni graf nas podsjeća da imamo podatke o mjesecima samo od kolovoza do prosinca. Vjerojatno nam treba više podataka kako bismo mogli donositi zaključke na linearan način.

Pogledajmo ponovno naše podatke za modeliranje:


In [ ]:
# Display first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Što ako bismo željeli predvidjeti `cijenu` bundeve na temelju stupaca `grad` ili `paket`, koji su tipa znakova? Ili još jednostavnije, kako bismo mogli pronaći korelaciju (koja zahtijeva da oba ulaza budu numerička) između, recimo, `paket` i `cijena`? 🤷🤷

Modeli strojnog učenja najbolje funkcioniraju s numeričkim značajkama, a ne tekstualnim vrijednostima, pa je općenito potrebno pretvoriti kategorijske značajke u numeričke reprezentacije.

To znači da moramo pronaći način za preoblikovanje naših prediktora kako bismo ih učinili lakšima za učinkovito korištenje modela, proces poznat kao `inženjering značajki`.


## 3. Predprocesiranje podataka za modeliranje s recipes 👩‍🍳👨‍🍳

Aktivnosti koje preoblikuju vrijednosti prediktora kako bi ih model mogao učinkovitije koristiti nazivaju se `inženjering značajki`.

Različiti modeli imaju različite zahtjeve za predprocesiranje. Na primjer, metoda najmanjih kvadrata zahtijeva `kodiranje kategorijalnih varijabli` poput mjeseca, sorte i city_name. To jednostavno uključuje `prevođenje` stupca s `kategorijalnim vrijednostima` u jedan ili više `numeričkih stupaca` koji zamjenjuju originalni.

Na primjer, pretpostavimo da vaši podaci uključuju sljedeću kategorijalnu značajku:

|  grad   |
|:-------:|
| Denver  |
| Nairobi |
|  Tokio  |

Možete primijeniti *ordinalno kodiranje* kako biste zamijenili svaku kategoriju jedinstvenom cjelobrojnom vrijednošću, ovako:

| grad |
|:----:|
|  0   |
|  1   |
|  2   |

I upravo to ćemo učiniti s našim podacima!

U ovom dijelu istražit ćemo još jedan fantastičan Tidymodels paket: [recipes](https://tidymodels.github.io/recipes/) - koji je osmišljen kako bi vam pomogao u predprocesiranju podataka **prije** treniranja vašeg modela. U svojoj srži, recept je objekt koji definira koje korake treba primijeniti na skup podataka kako bi bio spreman za modeliranje.

Sada, kreirajmo recept koji priprema naše podatke za modeliranje zamjenom jedinstvenog cijelog broja za sve opažanja u stupcima prediktora:


In [ ]:
# Specify a recipe
pumpkins_recipe <- recipe(price ~ ., data = new_pumpkins) %>% 
  step_integer(all_predictors(), zero_based = TRUE)


# Print out the recipe
pumpkins_recipe

Sjajno! 👏 Upravo smo kreirali naš prvi recept koji specificira ishod (cijenu) i odgovarajuće prediktore te da svi stupci prediktora trebaju biti kodirani u skup cijelih brojeva 🙌! Hajdemo to brzo razložiti:

-   Poziv funkcije `recipe()` s formulom govori receptu *uloge* varijabli koristeći podatke `new_pumpkins` kao referencu. Na primjer, stupac `price` dodijeljen je ulozi `outcome`, dok su ostali stupci dodijeljeni ulozi `predictor`.

-   `step_integer(all_predictors(), zero_based = TRUE)` specificira da svi prediktori trebaju biti pretvoreni u skup cijelih brojeva, pri čemu numeriranje počinje od 0.

Sigurni smo da možda razmišljate nešto poput: "Ovo je tako super!! Ali što ako trebam potvrditi da recepti rade točno ono što očekujem? 🤔"

To je sjajna misao! Vidite, jednom kada definirate svoj recept, možete procijeniti parametre potrebne za stvarnu predobradu podataka, a zatim izvući obrađene podatke. Obično to ne trebate raditi kada koristite Tidymodels (vidjet ćemo uobičajenu konvenciju za trenutak -\> `workflows`), ali može biti korisno kada želite napraviti neku vrstu provjere kako biste potvrdili da recepti rade ono što očekujete.

Za to će vam trebati još dva glagola: `prep()` i `bake()`, a kao i uvijek, naši mali R prijatelji od strane [`Allison Horst`](https://github.com/allisonhorst/stats-illustrations) pomoći će vam da to bolje razumijete!

<p >
   <img src="../../images/recipes.png"
   width="550"/>
   <figcaption>Ilustracija @allison_horst</figcaption>


<!--![Ilustracija \@allison_horst](../../../../../../2-Regression/3-Linear/images/recipes.png){width="550"}-->


[`prep()`](https://recipes.tidymodels.org/reference/prep.html): procjenjuje potrebne parametre iz skupa za treniranje koji se kasnije mogu primijeniti na druge skupove podataka. Na primjer, za određeni stupac prediktora, koja će se opažanja dodijeliti kao cijeli broj 0, 1, 2 itd.

[`bake()`](https://recipes.tidymodels.org/reference/bake.html): uzima pripremljeni recept i primjenjuje operacije na bilo koji skup podataka.

Dakle, pripremimo i primijenimo naše recepte kako bismo zaista potvrdili da će se stupci prediktora prvo kodirati prije nego što se model prilagodi.


In [ ]:
# Prep the recipe
pumpkins_prep <- prep(pumpkins_recipe)

# Bake the recipe to extract a preprocessed new_pumpkins data
baked_pumpkins <- bake(pumpkins_prep, new_data = NULL)

# Print out the baked data set
baked_pumpkins %>% 
  slice_head(n = 10)

Woo-hoo!🥳 Obradeni podaci `baked_pumpkins` imaju sve prediktore kodirane, što potvrđuje da će koraci predobrade definirani kao naš recept funkcionirati kako je očekivano. To ih čini teže čitljivima za vas, ali mnogo razumljivijima za Tidymodels! Odvojite malo vremena da otkrijete koja je opažanja mapirana na odgovarajući cijeli broj.

Također je vrijedno spomenuti da je `baked_pumpkins` podatkovni okvir na kojem možemo provoditi izračune.

Primjerice, pokušajmo pronaći dobru korelaciju između dvije točke vaših podataka kako bismo potencijalno izgradili dobar prediktivni model. Koristit ćemo funkciju `cor()` za to. Upišite `?cor()` kako biste saznali više o funkciji.


In [ ]:
# Find the correlation between the city_name and the price
cor(baked_pumpkins$city_name, baked_pumpkins$price)

# Find the correlation between the package and the price
cor(baked_pumpkins$package, baked_pumpkins$price)


Kako se ispostavilo, postoji samo slaba povezanost između Grada i Cijene. Međutim, postoji nešto bolja povezanost između Paketa i njegove Cijene. To ima smisla, zar ne? Obično, što je veća kutija proizvoda, to je viša cijena.

Dok smo na toj temi, pokušajmo također vizualizirati matricu korelacije svih stupaca koristeći paket `corrplot`.


In [ ]:
# Load the corrplot package
library(corrplot)

# Obtain correlation matrix
corr_mat <- cor(baked_pumpkins %>% 
                  # Drop columns that are not really informative
                  select(-c(low_price, high_price)))

# Make a correlation plot between the variables
corrplot(corr_mat, method = "shade", shade.col = NA, tl.col = "black", tl.srt = 45, addCoef.col = "black", cl.pos = "n", order = "original")

🤩🤩 Puno bolje.

Dobro pitanje koje sada možemo postaviti o ovim podacima je: '`Koju cijenu mogu očekivati za određeni paket bundeva?`' Idemo odmah na to!

> Napomena: Kada **`bake()`** pripremljeni recept **`pumpkins_prep`** s **`new_data = NULL`**, dobivate obrađene (tj. kodirane) podatke za treniranje. Ako imate drugi skup podataka, primjerice testni skup, i želite vidjeti kako bi recept obradio te podatke, jednostavno biste ispekli **`pumpkins_prep`** s **`new_data = test_set`**.

## 4. Izgradnja modela linearne regresije

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografika od Dasani Madipalli</figcaption>


<!--![Infografika od Dasani Madipalli](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->


Sada kada smo izradili recept i zapravo potvrdili da će podaci biti pravilno predobrađeni, izgradimo regresijski model kako bismo odgovorili na pitanje: `Koju cijenu mogu očekivati za određeni paket bundeve?`

#### Treniranje linearnog regresijskog modela koristeći skup za treniranje

Kao što ste možda već zaključili, stupac *price* je `izlazna` varijabla, dok je stupac *package* `prediktorska` varijabla.

Da bismo to postigli, prvo ćemo podijeliti podatke tako da 80% ide u skup za treniranje, a 20% u testni skup. Zatim ćemo definirati recept koji će kodirati prediktorski stupac u niz cijelih brojeva, a potom izraditi specifikaciju modela. Nećemo pripremati i peći naš recept jer već znamo da će pravilno predobraditi podatke.


In [ ]:
set.seed(2056)
# Split the data into training and test sets
pumpkins_split <- new_pumpkins %>% 
  initial_split(prop = 0.8)


# Extract training and test data
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)



# Create a recipe for preprocessing the data
lm_pumpkins_recipe <- recipe(price ~ package, data = pumpkins_train) %>% 
  step_integer(all_predictors(), zero_based = TRUE)



# Create a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")

Bravo! Sada kada imamo recept i specifikaciju modela, trebamo pronaći način da ih spojimo u objekt koji će prvo unaprijed obraditi podatke (prep+bake iza kulisa), prilagoditi model na unaprijed obrađenim podacima i također omogućiti potencijalne aktivnosti post-obrade. Kako ti se to čini za mir u duši!🤩

U Tidymodels-u, ovaj praktični objekt zove se [`workflow`](https://workflows.tidymodels.org/) i zgodno drži tvoje komponente modeliranja! Ovo bismo nazvali *pipelines* u *Pythonu*.

Dakle, hajdemo sve spojiti u workflow!📦


In [ ]:
# Hold modelling components in a workflow
lm_wf <- workflow() %>% 
  add_recipe(lm_pumpkins_recipe) %>% 
  add_model(lm_spec)

# Print out the workflow
lm_wf

Uz to, tijek rada može se prilagoditi/trenirati na gotovo isti način kao i model.


In [ ]:
# Train the model
lm_wf_fit <- lm_wf %>% 
  fit(data = pumpkins_train)

# Print the model coefficients learned 
lm_wf_fit

Iz modela možemo vidjeti koeficijente naučene tijekom treniranja. Oni predstavljaju koeficijente pravca najbolje prilagodbe koji nam daje najmanju ukupnu pogrešku između stvarne i predviđene varijable.

#### Procjena performansi modela pomoću testnog skupa

Vrijeme je da vidimo kako se model pokazao 📏! Kako to možemo učiniti?

Sada kada smo istrenirali model, možemo ga koristiti za predviđanja na testnom_skupu koristeći `parsnip::predict()`. Zatim možemo usporediti ta predviđanja sa stvarnim vrijednostima oznaka kako bismo procijenili koliko dobro (ili ne!) model funkcionira.

Krenimo s izradom predviđanja za testni skup, a zatim spojimo stupce s testnim skupom.


In [ ]:
# Make predictions for the test set
predictions <- lm_wf_fit %>% 
  predict(new_data = pumpkins_test)


# Bind predictions to the test set
lm_results <- pumpkins_test %>% 
  select(c(package, price)) %>% 
  bind_cols(predictions)


# Print the first ten rows of the tibble
lm_results %>% 
  slice_head(n = 10)

Da, upravo ste trenirali model i koristili ga za predviđanja!🔮 Je li dobar? Hajdemo procijeniti izvedbu modela!

U Tidymodels-u to radimo pomoću `yardstick::metrics()`! Za linearnu regresiju, usredotočimo se na sljedeće metrike:

-   `Root Mean Square Error (RMSE)`: Kvadratni korijen [MSE](https://en.wikipedia.org/wiki/Mean_squared_error). Ovo daje apsolutnu metriku u istoj jedinici kao i oznaka (u ovom slučaju, cijena bundeve). Što je manja vrijednost, to je model bolji (u pojednostavljenom smislu, predstavlja prosječnu cijenu za koju su predviđanja pogrešna!).

-   `Coefficient of Determination (obično poznat kao R-squared ili R2)`: Relativna metrika u kojoj je veća vrijednost bolja za prilagodbu modela. U suštini, ova metrika predstavlja koliko varijance između predviđenih i stvarnih vrijednosti oznake model može objasniti.


In [ ]:
# Evaluate performance of linear regression
metrics(data = lm_results,
        truth = price,
        estimate = .pred)

Evo kako izgleda izvedba modela. Pogledajmo možemo li dobiti bolji uvid vizualizacijom raspršenog dijagrama paketa i cijene, a zatim koristiti predviđanja za dodavanje linije najboljeg pristajanja.

To znači da ćemo morati pripremiti i obraditi testni skup kako bismo kodirali stupac paketa, a zatim ga povezati s predviđanjima našeg modela.


In [ ]:
# Encode package column
package_encode <- lm_pumpkins_recipe %>% 
  prep() %>% 
  bake(new_data = pumpkins_test) %>% 
  select(package)


# Bind encoded package column to the results
lm_results <- lm_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
lm_results %>% 
  slice_head(n = 5)


# Make a scatter plot
lm_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "orange", size = 1.2) +
  xlab("package")
  


Super! Kao što možete vidjeti, model linearne regresije ne generalizira baš dobro odnos između paketa i njegove odgovarajuće cijene.

🎃 Čestitamo, upravo ste stvorili model koji može pomoći u predviđanju cijene nekoliko vrsta bundeva. Vaš blagdanski vrt s bundevama bit će prekrasan. No, vjerojatno možete stvoriti još bolji model!

## 5. Izgradite model polinomne regresije

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografika: Dasani Madipalli</figcaption>


<!--![Infografika: Dasani Madipalli](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->


Ponekad naši podaci možda nemaju linearni odnos, ali i dalje želimo predvidjeti ishod. Polinomna regresija može nam pomoći u predviđanju složenijih nelinearnih odnosa.

Uzmimo za primjer odnos između paketa i cijene u našem skupu podataka o bundevama. Iako ponekad postoji linearni odnos između varijabli - što je bundeva veća po volumenu, to je cijena viša - ponekad se ti odnosi ne mogu prikazati kao ravnina ili ravna linija.

> ✅ Evo [nekoliko dodatnih primjera](https://online.stat.psu.edu/stat501/lesson/9/9.8) podataka koji bi mogli koristiti polinomnu regresiju
>
> Pogledajte ponovno odnos između sorte i cijene na prethodnom grafu. Čini li se da bi ovaj dijagram raspršenja nužno trebao biti analiziran ravnom linijom? Možda ne. U tom slučaju možete pokušati s polinomnom regresijom.
>
> ✅ Polinomi su matematički izrazi koji mogu sadržavati jednu ili više varijabli i koeficijenata

#### Trenirajte model polinomne regresije koristeći skup za treniranje

Polinomna regresija stvara *zakrivljenu liniju* kako bi bolje odgovarala nelinijarnim podacima.

Pogledajmo hoće li polinomni model bolje predviđati rezultate. Slijedit ćemo postupak sličan onome koji smo koristili ranije:

-   Kreirajte recept koji specificira korake predobrade koje treba provesti na našim podacima kako bi bili spremni za modeliranje, tj. kodiranje prediktora i izračunavanje polinoma stupnja *n*

-   Izradite specifikaciju modela

-   Spojite recept i specifikaciju modela u tijek rada

-   Kreirajte model prilagođavanjem tijeka rada

-   Procijenite koliko dobro model radi na testnim podacima

Krenimo odmah!


In [ ]:
# Specify a recipe
poly_pumpkins_recipe <-
  recipe(price ~ package, data = pumpkins_train) %>%
  step_integer(all_predictors(), zero_based = TRUE) %>% 
  step_poly(all_predictors(), degree = 4)


# Create a model specification
poly_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")


# Bundle recipe and model spec into a workflow
poly_wf <- workflow() %>% 
  add_recipe(poly_pumpkins_recipe) %>% 
  add_model(poly_spec)


# Create a model
poly_wf_fit <- poly_wf %>% 
  fit(data = pumpkins_train)


# Print learned model coefficients
poly_wf_fit

  

#### Procjena performansi modela

👏👏Izradili ste polinomni model, hajde da napravimo predikcije na testnom skupu!


In [ ]:
# Make price predictions on test data
poly_results <- poly_wf_fit %>% predict(new_data = pumpkins_test) %>% 
  bind_cols(pumpkins_test %>% select(c(package, price))) %>% 
  relocate(.pred, .after = last_col())


# Print the results
poly_results %>% 
  slice_head(n = 10)

Woo-hoo, hajdemo procijeniti kako se model pokazao na testnom skupu koristeći `yardstick::metrics()`.


In [ ]:
metrics(data = poly_results, truth = price, estimate = .pred)

🤩🤩 Mnogo bolja izvedba.

`rmse` se smanjio s otprilike 7 na otprilike 3, što ukazuje na smanjenu pogrešku između stvarne cijene i predviđene cijene. Ovo možete *približno* protumačiti kao da su, u prosjeku, netočne prognoze pogrešne za oko 3 USD. `rsq` se povećao s otprilike 0.4 na 0.8.

Svi ovi pokazatelji ukazuju na to da polinomni model radi puno bolje od linearnog modela. Odlično obavljeno!

Pogledajmo možemo li to vizualizirati!


In [ ]:
# Bind encoded package column to the results
poly_results <- poly_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
poly_results %>% 
  slice_head(n = 5)


# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "midnightblue", size = 1.2) +
  xlab("package")


Možete vidjeti zakrivljenu liniju koja bolje odgovara vašim podacima! 🤩

Možete je učiniti još glađom tako da proslijedite polinomsku formulu funkciji `geom_smooth` ovako:


In [ ]:
# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_smooth(method = lm, formula = y ~ poly(x, degree = 4), color = "midnightblue", size = 1.2, se = FALSE) +
  xlab("package")

Baš poput glatke krivulje!🤩

Evo kako možete napraviti novu predikciju:


In [ ]:
# Make a hypothetical data frame
hypo_tibble <- tibble(package = "bushel baskets")

# Make predictions using linear model
lm_pred <- lm_wf_fit %>% predict(new_data = hypo_tibble)

# Make predictions using polynomial model
poly_pred <- poly_wf_fit %>% predict(new_data = hypo_tibble)

# Return predictions in a list
list("linear model prediction" = lm_pred, 
     "polynomial model prediction" = poly_pred)


Predviđanje pomoću `polynomial model` ima smisla, s obzirom na raspršene grafikone `price` i `package`! I, ako je ovo bolji model od prethodnog, gledajući iste podatke, trebate planirati budžet za ove skuplje bundeve!

🏆 Bravo! Napravili ste dva regresijska modela u jednoj lekciji. U završnom dijelu o regresiji naučit ćete o logističkoj regresiji za određivanje kategorija.

## **🚀Izazov**

Testirajte nekoliko različitih varijabli u ovoj bilježnici kako biste vidjeli kako korelacija utječe na točnost modela.

## [**Kviz nakon predavanja**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/14/)

## **Pregled i samostalno učenje**

U ovoj lekciji smo naučili o linearnoj regresiji. Postoje i drugi važni tipovi regresije. Pročitajte o tehnikama Stepwise, Ridge, Lasso i Elasticnet. Dobar tečaj za dodatno učenje je [Stanfordov tečaj o statističkom učenju](https://online.stanford.edu/courses/sohs-ystatslearning-statistical-learning).

Ako želite saznati više o korištenju nevjerojatnog Tidymodels okvira, pogledajte sljedeće resurse:

-   Tidymodels web stranica: [Počnite s Tidymodels](https://www.tidymodels.org/start/)

-   Max Kuhn i Julia Silge, [*Tidy Modeling with R*](https://www.tmwr.org/)*.*

###### **HVALA:**

[Allison Horst](https://twitter.com/allison_horst?lang=en) za stvaranje nevjerojatnih ilustracija koje čine R pristupačnijim i zanimljivijim. Pronađite više ilustracija u njenoj [galeriji](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).



---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden pomoću AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati autoritativnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane čovjeka. Ne preuzimamo odgovornost za nesporazume ili pogrešna tumačenja koja mogu proizaći iz korištenja ovog prijevoda.
